In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
import xarray as xr
import matplotlib.pyplot as plt

In [ ]:
import os
import netCDF4 as nc4
import shutil
import warnings
warnings.filterwarnings('ignore')

In [2]:
import xarray as xr
import xesmf as xe
import cartopy.crs as ccrs

### GET OBS DATA

In [3]:
gppfile='/glade/work/rfisher/ILAMB_data/DATA/gpp/FLUXCOM/gpp.nc'
laifile='/glade/work/rfisher/FATES_calibration/ILAMB_data/DATA/lai/MODIS/lai_0.5x0.5.nc'

In [4]:
ds_gpp = xr.open_dataset(gppfile)
ds_lai = xr.open_dataset(laifile)

ds_gpp=ds_gpp.mean('time')
ds_lai=ds_lai.mean('time')
ds_lai['lat']=ds_lai.lat*-1

In [5]:
ds=ds_gpp

### Get FATES output

#### read fates data

In [7]:
nruns=4
runs=['4x5_out_of_the_box_ann','test_SP_cal_4x5','test_SP_4x5_rosieLHC_2','test_NOCOMP_4x5_rosieLHC_2','test_CLM60SP_4x5','test_SP_4x5_rosieLHC_1',]
arcr = [1,1,1,1]
fatesrun=[1,1,1,1]
ychoose=[2006, 2006, 2006, 2029]

In [ ]:

output_data=[]
for n in range(nruns):
    run=runs[n]
    root='/glade/derecho/scratch/rfisher/'
    if(arcr[n]==1):
        filedir=root+'archive/'+run+'/lnd/hist/'
    else:
        filedir=root+run+'/run/'
    file=run+'.clm2.h0.'+str(ychoose[n])+'-*'
    filename=filedir+file
    print(n,filename)
    outd=xr.open_mfdataset(filename)
    output_data.append(outd)


0 /glade/derecho/scratch/rfisher/archive/4x5_out_of_the_box_ann/lnd/hist/4x5_out_of_the_box_ann.clm2.h0.2006-*


#### extract variables

In [ ]:
ds_out_gpp_ts=[]
ds_out_lai_ts=[]
for r in range(nruns):
    print('r',r)
    if(fatesrun[r]==1):
        ds_out_gpp_ts.append(output_data[r].FATES_GPP*output_data[r].FATES_FRACTION)
        ds_out_lai_ts.append(output_data[r].FATES_LAI*output_data[r].FATES_FRACTION)
    else:   
        ds_out_gpp_ts.append(output_data[r].FPSN)
        ds_out_lai_ts.append(output_data[r].TLAI)

In [ ]:
ds_out_gpp=[]
ds_out_lai=[]
conv = 3600*24*1000
for r in range(nruns):
    print('r',r)
    ds_out_gpp.append(np.multiply(ds_out_gpp_ts[r].mean('time'),conv))
    ds_out_lai.append(ds_out_lai_ts[r].mean('time'))

### Make regridder

In [ ]:
regridder_gpp = xe.Regridder(ds, ds_out_gpp[0], 'bilinear', periodic=True)

In [ ]:
regridder_lai = xe.Regridder(ds, ds_out_lai[0], 'bilinear', periodic=True)

### do regridding of data

In [ ]:
gpp45deg = regridder_gpp(ds_gpp['gpp'])
lai45deg = regridder_lai(ds_lai['lai'])
lai45deg['lat']=lai45deg.lat*-1

In [ ]:
def make_bias_plot(bias,figname):
    fig = plt.figure(figsize=(10, 5))
    # Create a GeoAxes with the PlateCarree projection
    #ax = plt.axes(projection=ccrs.PlateCarree())
    
    ax = plt.axes(projection=ccrs.Robinson())
    
    # Plot the data on the map
    bias.plot(ax=ax, transform=ccrs.PlateCarree(),cmap='RdYlBu', vmin=-1.5, vmax=1.5)
    ax.set_title('')
    ax.set_title('')
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.coastlines()
    
    # Show the plot
    plt.show()
    plt.savefig(figname,bbox_inches='tight')



In [ ]:
def make_abs_plot(bias,figname,vmn,vmx):
    fig = plt.figure(figsize=(10, 5))
    # Create a GeoAxes with the PlateCarree projection
    #ax = plt.axes(projection=ccrs.PlateCarree())
    
    ax = plt.axes(projection=ccrs.Robinson())
    
    # Plot the data on the map
    bias.plot(ax=ax, transform=ccrs.PlateCarree(),cmap='viridis',vmin=vmn,vmax=vmx)
    ax.set_title('')
    ax.set_title('')
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.coastlines()
    
    # Show the plot
    plt.show()
    plt.savefig(figname,bbox_inches='tight')



In [ ]:
make_abs_plot(ds_out_nocomp_calib3_lai,'fates lai',0,2)

In [ ]:
bias_cont_lai = ds_out_lai[0]-lai45deg
make_bias_plot(bias_cont_lai,'bias_lai_0.png')

In [ ]:
bias_cont_lai = ds_out_lai[1]-lai45deg
make_bias_plot(bias_cont_lai,'bias_lai_1.png')

In [ ]:
bias_cont_lai = ds_out_lai[2]-lai45deg
make_bias_plot(bias_cont_lai,'bias_lai_1.png')

In [ ]:
bias_cont_lai = ds_out_lai[3]-lai45deg
make_bias_plot(bias_cont_lai,'bias_lai_1.png')

In [ ]:
bias_cont_lai = ds_out_gpp[0]-gpp45deg
make_bias_plot(bias_cont_lai,'bias_lai_1.png')

In [ ]:
bias_cont_lai = ds_out_gpp[1]-gpp45deg
make_bias_plot(bias_cont_lai,'bias_lai_1.png')

In [ ]:
bias_cont_lai = ds_out_gpp[2]-gpp45deg
make_bias_plot(bias_cont_lai,'bias_lai_1.png')

In [ ]:
bias_cont_lai = ds_out_gpp[3]-gpp45deg
make_bias_plot(bias_cont_lai,'bias_lai_1.png')

In [ ]:
bias_cont_lai = ds_out_nocomp_calib3_lai-lai45deg
make_bias_plot(bias_cont_lai,'bias_cont_lai_sp.png')

In [ ]:
make_abs_plot(ds_out_test_sp_gpp,'test_sp_gpp_abs.png',0,4)

In [ ]:
bias_calib1 = ds_out_sp_calib1_gpp-ds_out_sp_calib3_gpp
make_bias_plot(bias_calib1,'figs/bias_calib1_gpp.png')

In [ ]:
bias_cont_gpp.plot()

In [ ]:
plt.savefig('figtest.png')

### Look at PFT file differences

In [ ]:
p1='/glade/u/home/rfisher/rosiefork_fates_global_cal/rosie_ppe/nocomp_parameter_modification_python/intermediate_pfiles/fates_params_vertsc_dayl_SPcal_4.nc'

In [ ]:
p2='/glade/u/home/rfisher/rosiefork_fates_global_cal/rosie_ppe/nocomp_parameter_modification_python/paramfiles/SP_LHC//sp_lhc_v1_40.nc'

In [ ]:
p1d=xr.open_dataset(p1)
p2d=xr.open_dataset(p1)

In [ ]:
ds1 = xr.open_dataset(p1)
ds2 = xr.open_dataset(p2)

# Loop through the variables and compare them
for var in ds1.variables:
    if var in ds2.variables:
        if not ds1[var].equals(ds2[var]):
            print(f"Differences found in variable {var}")
            print('calib',ds1[var].values)
            print('pftsg',ds2[var].values)
    else:
        print(f"Variable {var} not found in {file2}")

# Check for variables in file2 that are not in file1
for var in ds2.variables:
    if var not in ds1.variables:
        print(f"Variable {var} not found in {file1}")

# Close the datasets
ds1.close()
ds2.close()